In [2]:
import ast
from openai import OpenAI
import pandas as pd
import numpy as np
from transformers import pipeline
import pycountry
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import pandas as pd
from tqdm import tqdm
import pycountry
import logging
import country_converter as coco

# Apply the function to the DataFrame with tqdm
tqdm.pandas()  # Initialize tqdm for pandas


In [12]:

client = OpenAI(api_key=api_key)
def ask_gpt(abstract):
    response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": f"% As a Name entity recognition tool, you are responsible for analyzing academic texts from various sources. Your task is to find all the country names mentioned in the abstract\\\\\n% Constraint: Answer with only the python list of countries in iso3 format \nmentioned in the text that is most accurate and nothing else.\\\\ \\\\\n% Abstract:{abstract} "
            }
          ]
        }
    
          ]
        ,
      temperature=0.3,
      max_tokens=2048,
      top_p=0.5,
      frequency_penalty=0,
      presence_penalty=0,
      response_format={
        "type": "text"
      }
    )
        # Strip the backticks and language identifier
    stripped = response.choices[0].message.content.strip("```").replace("python\n", "")
    
    # Convert the string to a Python object
    
    result = ast.literal_eval(stripped)
    return result

In [6]:
df=pd.read_csv(r'C:\Users\Yasaman\Arab_spring_scholarly_attention\Validating\final_annotated.csv')
df['Text']=df['Title']+' '+df['Abstract']
df['union_annotation'] = df['union_annotation'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df['intersection_annotation'] = df['intersection_annotation'].apply(lambda x: eval(x) if isinstance(x, str) else x)

In [13]:
df["locations"] = df["Text"].progress_apply(ask_gpt)

100%|██████████| 1000/1000 [11:06<00:00,  1.50it/s]


In [14]:
df["locations"]=df["locations"].apply(lambda x:[y.lower() for y in x])

In [15]:
print('overall accuracy (union)', sum(df['locations']==df['union_annotation'])/1000)
print('overall accuracy (intersection)', sum(df['locations']==df['intersection_annotation'])/1000)

overall accuracy (union) 0.812
overall accuracy (intersection) 0.78


In [16]:
sample_accuracies = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "accuracy_union": (x.apply(lambda row: set(row['locations']) == set(row['union_annotation']), axis=1).mean()),
        "accuracy_intersection": (x.apply(lambda row: set(row['locations']) == set(row['intersection_annotation']), axis=1).mean())
    })
).reset_index()

sample_accuracies = sample_accuracies.sort_values(by='accuracy_union').reset_index(drop=True)
sample_accuracies

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_33152\2343847047.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_accuracies = df.groupby("SampleGroup").apply(


,SampleGroup,accuracy_union,accuracy_intersection
0,with_mention,0.803571,0.728571
1,with_mention_arab,0.840000,0.790000
2,field_20,0.930769,0.901923


In [17]:
sample_accuracies = df.groupby("SampleGroup").apply(
    lambda x: pd.Series({
        "jaccard_union": (
            x.apply(
                lambda row: len(set(row['locations']).intersection(set(row['union_annotation']))) /
                            len(set(row['locations']).union(set(row['union_annotation'])))
                if len(set(row['locations']).union(set(row['union_annotation']))) > 0 else 1,
                axis=1
            ).mean()
        ),
        "jaccard_intersection": (
            x.apply(
                lambda row: len(set(row['locations']).intersection(set(row['intersection_annotation']))) /
                            len(set(row['locations']).union(set(row['intersection_annotation'])))
                if len(set(row['locations']).union(set(row['intersection_annotation']))) > 0 else 1,
                axis=1
            ).mean()
        )
    })
).reset_index()

sample_accuracies = sample_accuracies.sort_values(by='jaccard_union').reset_index(drop=True)
sample_accuracies

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_33152\388812476.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_accuracies = df.groupby("SampleGroup").apply(


,SampleGroup,jaccard_union,jaccard_intersection
0,with_mention,0.845103,0.778469
1,with_mention_arab,0.913708,0.855610
2,field_20,0.934982,0.907418


In [18]:
df.to_csv(r'C:\Users\Yasaman\Arab_spring_scholarly_attention\Validating\gpt.csv', index=False)